In [125]:
import re
import ast
import numpy as np
import math
from functools import cache
from itertools import product

def read_file_to_string(file_path):
    with open(file_path,'r') as f:
        return f.read()
    
day_19_string = read_file_to_string('day_19_input.txt')
day_19_test = read_file_to_string('day_19_test.txt')
#day_19_test_2 = read_file_to_string('day_19_test_2.txt')
# day_19_test_3 = read_file_to_string('day_19_test_3.txt')
# day_19_test_4 = read_file_to_string('day_19_test_4.txt')
# day_19_test_5 = read_file_to_string('day_19_test_5.txt')

In [126]:
def parse_string(string):

    day_19=string.split('\n\n')
    rules = day_19[0].split('\n')
    inputs = day_19[1].split('\n')

    rules = {string[:string.index('{')]:string[string.index('{')+1:-1].split(',') for string in rules}
    
    inputs = [{re.findall('[xmas]',input)[j]: re.findall('[0-9]+',input)[j] for j in range(4)} for input in inputs]
    for dic in inputs:
        for k,v in dic.items():
            dic[k] = int(v)
    

    return rules, inputs

In [127]:
test_rules, test_parts = parse_string(day_19_test)

TypeError: 'dict' object is not callable

In [24]:
def parse_rule(rule_string):
    
    
    rule = re.findall(r'<|>', rule_string)[0]
    letter = rule_string[0]
    number = int(re.findall(r'[0-9]+', rule_string)[0])
    output = rule_string[rule_string.rindex(':')+1:]

    return (letter, rule, number, output)

In [25]:
parse_rule('a<2006:qkq')

('a', '<', 2006, 'qkq')

In [49]:
def accepted_or_rejected(part_dict, rules,start='in'):

    place = start

    #print(place)
    while True:
        for rule in rules[place]:
            #print(rule)
            if rule == 'A':
                return True
            elif rule == 'R':
                return False
            elif rule in rules.keys():
                place = rule
                break
            else:
                letter, ineq, number, output = parse_rule(rule)

                if ineq == '<':
                    if part_dict[letter] < number:
                        if output == 'A':
                            return True
                        elif output == 'R':
                            return False
                        else:
                            place = output
                            break



                elif ineq == '>':
                    if part_dict[letter] > number:
                        if output == 'A':
                            return True
                        elif output == 'R':
                            return False
                        else:
                            place = output
                            break


In [53]:
def part_one(parts, rules):

    sum_ = 0
    for part in parts:
        if accepted_or_rejected(part, rules, start='in'):
            sum_ += sum(list(part.values()))

    return sum_

In [54]:
part_one(parts, rules)

19114

In [57]:
input_rules, input_parts = parse_string(day_19_string)

In [58]:
part_one(input_parts, input_rules)

476889

## Part 2

In [128]:
def parse_string_2(string):

    day_19=string.split('\n\n')
    rules = day_19[0].split('\n')

    rules = {string[:string.index('{')]:string[string.index('{')+1:-1].split(',') for string in rules}
    return rules

In [62]:
def parse_rule_2(rule_string):
    
    
    rule = re.findall(r'<|>', rule_string)[0]
    letter = rule_string[0]
    number = int(re.findall(r'[0-9]+', rule_string)[0])
    output = rule_string[rule_string.rindex(':')+1:]

    condition, opposite = ((letter, rule, number, output), (letter, '<=' if rule == '>' else '>=', number))

    return condition, opposite

In [130]:
test_rules = parse_string_2(day_19_test)
input_rules = parse_string_2(day_19_string)

In [135]:
def accepted_or_rejected_ranges(rules):

    #we get to a rule in our process, add two parts to the part dict: one where we follow the rule and another where we don't
    #we want to have 8 eles to the parts dict, max_a and min_a etc.

    parts_dict = {'max_x':4000,
                  'min_x':1,
                  'max_m':4000,
                  'min_m':1,
                  'max_a':4000,
                  'min_a':1,
                  'max_s':4000,
                  'min_s':1
                  }
    
    #parts_dict_opposite = parts_dict.copy()

    finished_processes = [] #put a hard copy of the done dict here
    
    queue = [(parts_dict,'in',0)]



    #print(place)
    while len(queue) > 0:
        eg_dict, key, index = queue.pop(0)
        #print(eg_dict, key, index)

        rule = rules[key][index]
        #print(rule)
        if rule == 'A':
            #print(key)
            finished_processes.append(eg_dict)
        elif rule == 'R':
            #rejected, don't need to store it anywhere (could delete the dict to save memory?)
            continue
        elif rule in rules.keys():
            #need to push to queue
            queue.append((eg_dict,rule,0))
            continue
        else:
            
            letter, ineq, number, output = parse_rule_2(rule)[0]

            if ineq == '<':
                parts_dict_follows = eg_dict.copy()
                parts_dict_opposite = eg_dict.copy()

                parts_dict_follows['max_'+letter] = number-1 if number-1 < parts_dict_follows['max_'+letter] else parts_dict_follows['max_'+letter]
                parts_dict_opposite['min_'+letter] = number if number > parts_dict_opposite['min_'+letter] else parts_dict_opposite['min_'+letter]
                
                
                if output == 'A':
                    #print(key)
                    finished_processes.append(parts_dict_follows)
                elif output == 'R':
                    queue.append((parts_dict_opposite,key,index+1))
                    continue
                else:
                    queue.append((parts_dict_follows,output,0))
                
                queue.append((parts_dict_opposite,key,index+1))

                    


            elif ineq == '>':
                parts_dict_follows = eg_dict.copy()
                parts_dict_opposite = eg_dict.copy()

                parts_dict_follows['min_'+letter] = number+1 if number+1 > parts_dict_follows['min_'+letter] else parts_dict_follows['min_'+letter]
                parts_dict_opposite['max_'+letter] = number if number < parts_dict_opposite['max_'+letter] else parts_dict_opposite['max_'+letter]
                
                if output == 'A':
                    #print(key)
                    finished_processes.append(parts_dict_follows)
                elif output == 'R':
                    queue.append((parts_dict_opposite,key,index+1))
                    continue
                else:
                    queue.append((parts_dict_follows,output,0))

                
                queue.append((parts_dict_opposite,key,index+1))

        #print(queue)

    return finished_processes
#after editing it, store this in the queue with the rule in dictionary AND the index of the dict value so we know which rule to get to next


In [136]:
def part_two(ranges):
    sum_ = 0

    for range in ranges:

        range_product = (1+range['max_x']-range['min_x'])*(1+range['max_m']-range['min_m'])*(1+range['max_a']-range['min_a'])*(1+range['max_s']-range['min_s'])
        sum_ += range_product if range_product > 0 else 0

    return sum_
#currently missing ranges!

In [137]:
test_ranges=accepted_or_rejected_ranges(test_rules)

In [138]:
part_two(test_ranges)

167409079868000

In [139]:
input_ranges = accepted_or_rejected_ranges(input_rules)
part_two(input_ranges)

132380153677887